In [3]:
#!pip install --upgrade azureml-sdk azureml-dataprep matplotlib

In [41]:
#Get AzureML Workspace config
#Setup environment using samples here: https://github.com/Azure/MachineLearningNotebooks/tree/master/setup-environment
from azureml.core import Workspace, Dataset
workspace = Workspace.from_config()
#print(workspace)

In [4]:
#Avro files location pre-registered as a files dataset. Ensure the dataset path is of the format folder/** to pick up all partitions and folders inside.
dataset_name="eventhub-capture-avro"
dataset = Dataset.get_by_name(workspace, name=dataset_name)
print(dataset.name)


eventhub-capture-avro


In [38]:
# mount dataset manually instead of using
# with dataset.mount() as mount_context:
import os
mount_context=dataset.mount()
mount_context.start()
# See where we mounted the file dataset
print(mount_context.mount_point)
# List files in there
print(os.listdir(mount_context.mount_point))

/tmp/tmpdip7okeh
['eventhub1']


In [39]:
#Display Recursive - folder and files 
dataset_mount_folder = mount_context.mount_point
for root,d_names,f_names in os.walk(dataset_mount_folder):
	print(root, d_names, f_names)

/tmp/tmpdip7okeh ['eventhub1'] []
/tmp/tmpdip7okeh/eventhub1 ['0'] []
/tmp/tmpdip7okeh/eventhub1/0 ['2021'] []
/tmp/tmpdip7okeh/eventhub1/0/2021 ['09'] []
/tmp/tmpdip7okeh/eventhub1/0/2021/09 ['08'] []
/tmp/tmpdip7okeh/eventhub1/0/2021/09/08 ['06', '07', '08'] []
/tmp/tmpdip7okeh/eventhub1/0/2021/09/08/06 ['35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59'] []
/tmp/tmpdip7okeh/eventhub1/0/2021/09/08/06/35 [] ['14.avro']
/tmp/tmpdip7okeh/eventhub1/0/2021/09/08/06/36 [] ['14.avro']
/tmp/tmpdip7okeh/eventhub1/0/2021/09/08/06/37 [] ['14.avro']
/tmp/tmpdip7okeh/eventhub1/0/2021/09/08/06/38 [] ['14.avro']
/tmp/tmpdip7okeh/eventhub1/0/2021/09/08/06/39 [] ['14.avro']
/tmp/tmpdip7okeh/eventhub1/0/2021/09/08/06/40 [] ['14.avro']
/tmp/tmpdip7okeh/eventhub1/0/2021/09/08/06/41 [] ['14.avro']
/tmp/tmpdip7okeh/eventhub1/0/2021/09/08/06/42 [] ['14.avro']
/tmp/tmpdip7okeh/eventhub1/0/2021/09/08/06/43 [] ['

In [40]:
#pickup a sample File
files = os.listdir(os.path.join(dataset_mount_folder,'eventhub1/0/2021/09/08/06/35'))
print(files)

['14.avro']


In [31]:
# !pip install fastavro
from fastavro import writer, reader, parse_schema

with open(os.path.join(dataset_mount_folder,'eventhub1/0/2021/09/08/06/35/14.avro'), 'rb') as fo:
    avro_reader = reader(fo)
    records = [r for r in avro_reader]
print(records)

[{'SequenceNumber': 1564641, 'Offset': '403726925824', 'EnqueuedTimeUtc': '9/8/2021 6:35:37 AM', 'SystemProperties': {'x-opt-partition-key': '466920403025604', 'x-opt-enqueued-time': 1631082937669}, 'Properties': {}, 'Body': b'{"RecordType":"MO","SystemIdentity":"d0","FileNum":"0","SwitchNum":"Australia","CallingNum":"789041222","CallingIMSI":"466920403025604","CalledNum":"123487581","CalledIMSI":"466920401237309","DateS":"20210908","TimeS":null,"TimeType":1,"CallPeriod":0,"CallingCellID":null,"CalledCellID":null,"ServiceType":"b","Transfer":1,"IncomingTrunk":null,"OutgoingTrunk":"609","MSRN":"886932428687","CalledNum2":null,"FCIFlag":null,"callrecTime":"2021-09-08T06:35:35Z"}'}, {'SequenceNumber': 1564642, 'Offset': '403726926384', 'EnqueuedTimeUtc': '9/8/2021 6:35:37 AM', 'SystemProperties': {'x-opt-partition-key': '262021390056324', 'x-opt-enqueued-time': 1631082937966}, 'Properties': {}, 'Body': b'{"RecordType":"MO","SystemIdentity":"d0","FileNum":"2","SwitchNum":"Germany","Calling

In [35]:
#Add records to a pandas DataFrame
import pandas as pd
df = pd.DataFrame.from_records(records)
df.head(2)

,SequenceNumber,Offset,EnqueuedTimeUtc,SystemProperties,Properties,Body
0,1564641,403726925824,9/8/2021 6:35:37 AM,"{'x-opt-partition-key': '466920403025604', 'x-...",{},"b'{""RecordType"":""MO"",""SystemIdentity"":""d0"",""Fi..."
1,1564642,403726926384,9/8/2021 6:35:37 AM,"{'x-opt-partition-key': '262021390056324', 'x-...",{},"b'{""RecordType"":""MO"",""SystemIdentity"":""d0"",""Fi..."


In [36]:
# Unmount dataset
mount_context.stop()